Requisitos

In [1]:
!pip install matplotlib
!pip install pandas

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

Resultados de los torneos cargados directamente desde GitHub

In [ ]:
nice_or_die_dom = "https://github.com/luiserh1/NGA/raw/main/log/dom1.csv"
grocery_dom = "https://github.com/luiserh1/NGA/raw/main/log/dom2.csv"
energy_dom = "https://github.com/luiserh1/NGA/raw/main/log/dom3.csv"

In [ ]:
dom1_df = pd.read_csv(nice_or_die_dom)
dom2_df = pd.read_csv(grocery_dom)
dom3_df = pd.read_csv(energy_dom)

Extraenos la información que nos interesa